In [2]:
from src.utils.spark_utils import get_spark_session

In [3]:
spark=get_spark_session()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/21 22:56:32 WARN Utils: Your hostname, vivekpanthi-Swift-SF514-55TA, resolves to a loopback address: 127.0.1.1; using 192.168.254.15 instead (on interface wlp0s20f3)
25/11/21 22:56:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/vivekpanthi/DataEngineering/E-commerce-Data-Pipeline/myvenv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/vivekpanthi/.ivy2.5.2/cache
The jars for the packages stored in: /home/vivekpanthi/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dc3f70ea-90ed-40ba-a624-e2d204afd285;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: 

In [10]:
df=spark.read.format("delta").load("data/bronze/bronze_customers_raw")

In [ ]:
df=spark.read.format("csv").option("header", True).load("data/landing/customers/customers.csv")

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df.filter(col("email").isNotNull()).show()

+-----------+----------+---------+--------------------+--------------------+-----------------+----------------+--------------+---------+-------------+----------------------+--------------+---------------+-----------------+--------------------+--------------------+--------------------+----------------+----------------------+---------------------------+--------------------+--------------------------+----------------------+--------------------------+--------------------+--------------------+---------------+
|customer_id|first_name|last_name|               email|               phone|registration_date|customer_segment|lifetime_value|is_active|_extra_fields|_dq_completeness_score|_dq_has_errors|_dq_error_count|_dq_warning_count|   _dq_error_details| _dq_warning_details| _bronze_record_hash|_dq_is_duplicate|_dq_duplicate_of_batch|_bronze_ingestion_timestamp| _bronze_source_file|_bronze_ingestion_batch_id|_bronze_ingestion_date|_bronze_source_line_number|  _bronze_raw_record|   _bronze_file_ha

In [16]:
BRONZE_TABLE_CUSTOMERS="bronze_ingestion_date"

In [10]:
df.select("payment_method").distinct().show()

+----------------+
|  payment_method|
+----------------+
|       gift_card|
|             cod|
|     Credit Card|
|          paypal|
|     CREDIT_CARD|
|          PayPal|
|     credit_card|
|          PAYPAL|
|      debit_card|
|Cash on Delivery|
|            NULL|
+----------------+



In [17]:
    df=spark.sql(f"""
        SELECT 
            customer_id,
            email,
            customer_segment,
            _dq_has_errors,
            _dq_error_count,
            _bronze_ingestion_timestamp
        FROM {BRONZE_TABLE_CUSTOMERS}
        ORDER BY _bronze_ingestion_timestamp DESC
        LIMIT 5
    """).show(truncate=False)

{"ts": "2025-11-17 01:19:09.756", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[TABLE_OR_VIEW_NOT_FOUND] The table or view `bronze_ingestion_date` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01", "context": {"errorClass": "TABLE_OR_VIEW_NOT_FOUND"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o37.sql.\n: org.apache.spark.sql.AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bronze_ingestion_date` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIE

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bronze_ingestion_date` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 9 pos 9;
'GlobalLimit 5
+- 'LocalLimit 5
   +- 'Sort ['_bronze_ingestion_timestamp DESC NULLS LAST], true
      +- 'Project ['customer_id, 'email, 'customer_segment, '_dq_has_errors, '_dq_error_count, '_bronze_ingestion_timestamp]
         +- 'UnresolvedRelation [bronze_ingestion_date], [], false


In [4]:
import os

bronze_base = "./data/bronze"

table_names = [
    d for d in os.listdir(bronze_base)
    if os.path.isdir(os.path.join(bronze_base, d))
]

print(table_names)


['bronze_customers_raw']


In [44]:

BRONZE_TABLE="bronze_order_items_raw"
BRONZE_ROOT_PATH=f"./data/bronze/bronze_order_items_raw"
BRONZE="bronze"
spark.sql(f"""
                CREATE TABLE IF NOT EXISTS default.{BRONZE_TABLE}
                USING DELTA
                LOCATION '{BRONZE_ROOT_PATH}'
            """)

DataFrame[]

In [34]:

BRONZE_TABLE="bronze_order_items_raw"
BRONZE="bronze"

In [23]:
from delta.tables import DeltaTable

In [69]:
DeltaTable.isDeltaTable(spark,BRONZE_ROOT_PATH)

True

In [45]:
spark.sql("SHOW TABLES IN default").show(truncate=False)


+---------+----------------------+-----------+
|namespace|tableName             |isTemporary|
+---------+----------------------+-----------+
|default  |bronze_order_items_raw|false      |
+---------+----------------------+-----------+



In [41]:
spark.table("bronze_order_items_raw").show()

AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `default`.`bronze_order_items_raw` doesn't exist.

In [46]:
spark.sql(f"SELECT * FROM default.bronze_order_items_raw")




AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `default`.`bronze_order_items_raw` doesn't exist.

In [27]:
spark.sql("SELECT * FROM default.bronze_order_items_raw LIMIT 10").show()

AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `default`.`bronze_order_items_raw` doesn't exist.

In [42]:
spark.sql("DESCRIBE DETAIL bronze_order_items_raw").show(truncate=False)


AnalysisException: [DELTA_PATH_DOES_NOT_EXIST] file:/home/vivekpanthi/DataEngineering/E-commerce-Data-Pipeline/spark-warehouse/data/bronze/bronze_order_items_raw doesn't exist, or is not a Delta table.;
DescribeDeltaDetailCommand
+- ResolvedTable org.apache.spark.sql.delta.catalog.DeltaCatalog@1357521f, default.bronze_order_items_raw, DeltaTableV2(org.apache.spark.sql.classic.SparkSession@50a9f278,file:/home/vivekpanthi/DataEngineering/E-commerce-Data-Pipeline/spark-warehouse/data/bronze/bronze_order_items_raw,Some(CatalogTable(
Catalog: spark_catalog
Database: default
Table: bronze_order_items_raw
Created Time: Fri Nov 21 14:54:25 NPT 2025
Last Access: UNKNOWN
Created By: Spark 4.0.1
Type: EXTERNAL
Provider: delta
Location: file:/home/vivekpanthi/DataEngineering/E-commerce-Data-Pipeline/spark-warehouse/data/bronze/bronze_order_items_raw
Partition Provider: Catalog)),Some(default.bronze_order_items_raw),None,Map())


In [43]:
spark.sql("SELECT * FROM default.bronze_order_items_raw").show()



AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `default`.`bronze_order_items_raw` doesn't exist.

In [71]:
existing_bronze=spark.read.format("delta").load("data/bronze/bronze_order_items_raw")

In [63]:
from pyspark.sql.functions import *

In [72]:
existing_bronze.agg(max("bronze_ingestion_date")).show(truncate=False)

+--------------------------+
|max(bronze_ingestion_date)|
+--------------------------+
|2025-11-21                |
+--------------------------+



In [58]:
from datetime import datetime

In [59]:
today_date=datetime.today().strftime("%Y-%m-%d")

In [60]:
today_date

'2025-11-21'